synchronize your pydoxtools directory:

- rclone sync pydoxtools/ xyntopia_gdrive:/pydoxtools -P --size-only --fast-list

In [ ]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
#import torch
#import torch_xla
#import torch_xla.core.xla_model as xm
#
#t = torch.randn(2, 2, device=xm.xla_device())
#print(t.device)
#print(t)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## setup pydoxtools code access, data & gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
PDX_DIR="/content/pydoxtools"

In [ ]:
%env TRAINING_DATA_DIR=/content/gdrive/MyDrive/pydoxtools/training_data

In [ ]:
!pip install faker evaluate transformers datasets accelerate nvidia-ml-py3

In [ ]:
!ls "/content/gdrive/My Drive/pydoxtools/pydoxtools"
!rm -r $PDX_DIR
!cp -r "/content/gdrive/My Drive/pydoxtools/pydoxtools" $PDX_DIR

The main issue when using google colab for our training is, that we only have python version 3.9 available, while pydoxtools currently needs 3.10 to function.and

because of this we are trying to only use the relevant parts of the library which is mainly the data generation part which is used for
the training.

In [ ]:
!rm $PDX_DIR/__init__.py
!touch $PDX_DIR/__init__.py
!pwd

finally...  run pydoxtools

In [ ]:
from pydoxtools import random_data_generators

## prepare training

In [ ]:
bg = random_data_generators.TextBlockGenerator.std_generator()
bg.classmap, bg.classmap_inv, bg.num_generators, bg.class_gen, bg.gen_mapping, bg.weights

In [ ]:
bg[10]

In [ ]:
from datasets import Dataset

dataset_size=10000

# generate datasets for finetuning
def my_gen():
    bgi = bg.__iter__()
    for i in range(1, dataset_size):
        text, label = next(bgi)
        yield {"label": label, "text":text}


dataset = Dataset.from_generator(my_gen)
dataset.set_format("torch")

In [ ]:
dataset

In [ ]:
model_name="sentence-transformers/all-MiniLM-L6-v2"
# model_name="bert-base-cased"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
train_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained(, num_labels=2).to("cuda")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

In [ ]:
from transformers import TrainingArguments

In [ ]:
import numpy as np
import evaluate

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## setup hardware

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
print_gpu_utilization()

In [ ]:
#!nvidia-smi

## run actual training

In [ ]:
default_args = {
    "output_dir": "test_trainer",
    "evaluation_strategy": "steps",
    "num_train_epochs": 10,
    "log_level": "error",
    "report_to": "none",
}

In [ ]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()


training_args = TrainingArguments(
    per_device_train_batch_size=8,
    #gradient_accumulation_steps=4,
    #gradient_checkpointing=True,
    **default_args)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    compute_metrics=compute_metrics,
)
#result = trainer.train()
#print_summary(result)

In [ ]:
#trainer.train()

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs